# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df_customer = data.groupby(['CustomerID','ProductName'])['Quantity'].sum().reset_index()

df_customer.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
table = pd.pivot_table(df_customer, values='Quantity', index=['ProductName'], columns=['CustomerID'],fill_value=0)

display(table.shape)
table.head()

(452, 1000)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [53]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))), 
                         index=table.columns, columns=table.columns)

display(distances.head())

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,NaN,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,NaN,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,NaN,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,NaN,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,NaN,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [54]:
table[669] = pd.Series(669)

similarities = distances[669].sort_values(ascending=False)[1:]

similarities = similarities.index[0:5]

similarities

Int64Index([33, 200, 264, 356, 412], dtype='int64', name='CustomerID')

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [55]:
def selecting(similar_customers):
    selecting_customers = df_customer.loc[(df_customer['CustomerID'] == similar_customers[0])|
                                      (df_customer['CustomerID'] == similar_customers[1])|
                                      (df_customer['CustomerID'] == similar_customers[2])|
                                      (df_customer['CustomerID'] == similar_customers[3])|
                                      (df_customer['CustomerID'] == similar_customers[4])]
    return selecting_customers

selecting(similarities).sort_values('Quantity', ascending= False).head()

,CustomerID,ProductName,Quantity
87,200,Lamb - Ground,3
21,33,Grouper - Fresh,2
145,264,Chocolate - Dark,2
25,33,Lettuce - Spring Mix,2
296,412,Pernod,2


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [56]:
ranked_products = selecting_customers.groupby(['ProductName']).agg({'Quantity':sum}).sort_values('Quantity', ascending=False)

type(ranked_products)

pandas.core.frame.DataFrame

In [57]:
ranked_products2 = (selecting_customers.groupby(['ProductName'])['Quantity'].sum().sort_values(ascending = False)).to_frame()


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [58]:
chosen_customer= df_customer.loc[(df_customer['CustomerID'] == 669)]
chosen_customer.shape

(60, 3)

In [59]:
chosen_customer= df_customer.loc[(df_customer['CustomerID'] == 669)]

merge_recomended = pd.merge(chosen_customer, ranked_products, how='right', on='ProductName')
merge_recomended = merge_recomended.fillna(0)

recommended_product = (merge_recomended.loc[(merge_recomended['Quantity_x'] == 0)]).sort_values(by = ['Quantity_x'], ascending = False)
recommended_product.head()

,CustomerID,ProductName,Quantity_x,Quantity_y
32,0.0,"Veal - Inside, Choice",0.0,4
159,0.0,Apricots - Dried,0.0,1
150,0.0,Coffee - Dark Roast,0.0,1
151,0.0,Cookies Cereal Nut,0.0,1
152,0.0,Cornflakes,0.0,1


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [60]:
#recomm_dict = {}      # recomm_dict['CustomerID'] = [product1, product2, product3, product4, product5]

customers_list= list(set(data['CustomerID']))

def recommender(customer):
    
    similarities = distances[customer].sort_values(ascending=False)[1:]
    similar_customers = similarities.index[0:5]
    def selecting(similar_customers):
        selecting_customers = df_customer.loc[(df_customer['CustomerID'] == similar_customers[0])|
                                      (df_customer['CustomerID'] == similar_customers[1])|
                                      (df_customer['CustomerID'] == similar_customers[2])|
                                      (df_customer['CustomerID'] == similar_customers[3])|
                                      (df_customer['CustomerID'] == similar_customers[4])]
        return selecting_customers
    ranked_products2 = (selecting(similar_customers).groupby(['ProductName'])['Quantity'].sum().sort_values(ascending = False)).to_frame()
    chosen_customer= df_customer.loc[(df_customer['CustomerID'] == customer)]
    merge_recomended = (pd.merge(chosen_customer, ranked_products, how='right', on='ProductName')).fillna(0)
    recommended_product = (merge_recomended.loc[(merge_recomended['Quantity_x'] == 0)]).sort_values(by = ['Quantity_x'], ascending = False)
    products = list(recommended_product['ProductName'])
    return products[0:5]

def form_dict(customer):
    recomm_dict[customer]=recommender(customer)
    return recomm_dict

recomm_dict = {}

for c in customers_list:
    recomm_dict.update(form_dict(c))
    
recomm_dict

{83973: ['Veal - Inside, Choice',
  'Crab - Imitation Flakes',
  'Chocolate - Dark',
  'Chocolate - Semi Sweet, Calets',
  'Cinnamon Buns Sticky'],
 59399: ['Sardines',
  'Cheese - Boursin, Garlic / Herbs',
  'Chocolate - Semi Sweet, Calets',
  'Cup - 6oz, Foam',
  'Clam Nectar'],
 92168: ['Veal - Inside, Choice',
  'Bandage - Flexible Neon',
  'Chocolate - Dark',
  'Chocolate - Semi Sweet, Calets',
  'Cup - 6oz, Foam'],
 49159: ['Veal - Inside, Choice',
  'Clam Nectar',
  'Coffee - Dark Roast',
  'Cookies Cereal Nut',
  'Cornflakes'],
 18441: ['Veal - Inside, Choice',
  'Bacardi Breezer - Tropical',
  'Cookies Cereal Nut',
  'Cornflakes',
  'Crab - Imitation Flakes'],
 22536: ['Veal - Inside, Choice',
  'Crackers - Trio',
  'Chocolate - Dark',
  'Chocolate - Semi Sweet, Calets',
  'Cup - 6oz, Foam'],
 86028: ['Veal - Inside, Choice',
  'Cake - Cake Sheet Macaroon',
  'Coffee - Dark Roast',
  'Cookies Cereal Nut',
  'Cornflakes'],
 75791: ['Veal - Inside, Choice',
  'Cheese - Wine',
  

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [62]:
df_recommend = (pd.DataFrame(recomm_dict)).T
df_recommend.columns = ['Product 1','Product 2','Product 3','Product 4','Product 5']

df_recommend.head()

,Product 1,Product 2,Product 3,Product 4,Product 5
83973,"Veal - Inside, Choice",Crab - Imitation Flakes,Chocolate - Dark,"Chocolate - Semi Sweet, Calets",Cinnamon Buns Sticky
59399,Sardines,"Cheese - Boursin, Garlic / Herbs","Chocolate - Semi Sweet, Calets","Cup - 6oz, Foam",Clam Nectar
92168,"Veal - Inside, Choice",Bandage - Flexible Neon,Chocolate - Dark,"Chocolate - Semi Sweet, Calets","Cup - 6oz, Foam"
49159,"Veal - Inside, Choice",Clam Nectar,Coffee - Dark Roast,Cookies Cereal Nut,Cornflakes
18441,"Veal - Inside, Choice",Bacardi Breezer - Tropical,Cookies Cereal Nut,Cornflakes,Crab - Imitation Flakes


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [68]:
# Diferentes distancias con un mismo cliente para ver si hay diferencias.
customer = 639

distances= pd.DataFrame(1/(1 + squareform(pdist(table.T, 'correlation'))), 
                         index=table.columns, columns=table.columns)
display ("Distance 'correlation': {}".format(recommender(customer)))

distances= pd.DataFrame(1/(1 + squareform(pdist(table.T, 'cityblock'))),
                        index=table.columns, columns=table.columns)
display ("Distance 'cityblock': {}".format(recommender(customer)))


distances= pd.DataFrame(1/(1 + squareform(pdist(table.T, 'cosine'))),
                        index=table.columns, columns=table.columns)
display ("Distance 'cosine': {}".format(recommender(customer)))


distances= pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))), 
                         index=table.columns, columns=table.columns)
display ("Distance 'euclidean': {}".format(recommender(customer)))


distances= pd.DataFrame(1/(1 + squareform(pdist(table.T, 'jaccard'))), 
                         index=table.columns, columns=table.columns)
display ("Distance 'jaccard': {}".format(recommender(customer)))

"Distance 'correlation': ['Wine - Ej Gallo Sierra Valley', 'Pepper - Black, Whole', 'Crackers - Trio', 'Cheese - Boursin, Garlic / Herbs', 'Cassis']"

"Distance 'cityblock': ['Wine - Ej Gallo Sierra Valley', 'Pepper - Black, Whole', 'Crackers - Trio', 'Cheese - Boursin, Garlic / Herbs', 'Cassis']"

"Distance 'cosine': ['Wine - Ej Gallo Sierra Valley', 'Pepper - Black, Whole', 'Crackers - Trio', 'Cheese - Boursin, Garlic / Herbs', 'Cassis']"

"Distance 'euclidean': ['Wine - Ej Gallo Sierra Valley', 'Pepper - Black, Whole', 'Crackers - Trio', 'Cheese - Boursin, Garlic / Herbs', 'Cassis']"

"Distance 'jaccard': ['Wine - Ej Gallo Sierra Valley', 'Pepper - Black, Whole', 'Crackers - Trio', 'Cheese - Boursin, Garlic / Herbs', 'Cassis']"

###  He probado de diferentes maneras poniendo diferentes distancias pero siempre salen las mismas recomendaciones. El dataframe de distancias es diferente.... no entiendo